# 임베딩 및 평가 기준별 정합성 계산 

In [24]:
import os
import sys
import torch
import pandas as pd
from langchain.schema import Document
from sentence_transformers import SentenceTransformer, models
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
# 채용공고 데이터 로드

job_opening = pd.read_csv('./data/job_opening.csv')

In [48]:
documents = [Document(page_content=f"{row['rec_idx']}: {row['jd_text']} {row['recruit_title']} {row['recruit_kewdcdnm']} {row['company_place']} {row['career']} {row['education']}", metadata={"id": idx}) for idx, row in job_opening.iterrows()]

In [ ]:
docs = [ document.page_content.replace("\n", " ").replace("채용공고 상세", " ") for document in documents ]

In [60]:
docs

["50245530:    [코스닥 상장사] 경영기획본부 신입2007년 12월 11일에 설립된 그 외 기타 의료용 기기 제조업업종의 피부미용 의료기기 제조,중개,무역사업을 하는 코스닥,중소기업,주식회사,외부감사법인,병역특례 인증업체,수출입 기업 입니다.모집부문 및 상세내용모집부문상세내용공통 자격요건ㆍ학력 : 대졸 이상 (4년)ㆍ학점 : 3.8 이상경영기획본부전략기획팀 1명주요업무담당업무ㆍ지원직무주요 회의체 운영 및 경영진 지원 (회의 안건 준비, 회의록 작성 등)조직별 성과 데이터 집계 및 분석, 경영진 보고사업 전략 및 중장기 계획 수립 지원기타 경영지원 및 프로젝트성 업무 수행지원자격데이터 분석 및 보고서 작성 능력 (Excel, PowerPoint 활용)커뮤니케이션 및 조율 역량 보유컨설팅 및 전략 기획 관련 경험 우대 ㆍ기타 필수 사항 우대사항✅ 전략기획, 경영관리 등 유관 업무 경험 보유자✅ 데이터 분석 및 리포팅 역량 우수자 (BI 툴, SQL 활용 가능자 우대)✅ 사업 전략 및 중장기 계획 수립 경험자✅ 주요 프로젝트 운영 경험 보유자근무조건ㆍ근무형태:정규직(수습기간)-3개월ㆍ근무일시:주 5일(월~금)ㆍ근무지역:(16827) 경기 용인시 수지구 신수로 767 분당수지 U-TOWER 19층(동천동) - 신분당선 동천 에서 200m 이내전형절차 서류전형 1차면접 2차면접(경우에 따라 생략 가능) 최종합격접수기간 및 방법ㆍ:2025년 3월 14일 (금) 14시~ 채용시ㆍ접수방법:사람인 입사지원ㆍ이력서양식:사람인 온라인 이력서ㆍ제출서류:유의사항ㆍ학력, 성별, 연령을 보지않는 블라인드 채용입니다. ㆍ입사지원 서류에 허위사실이 발견될 경우, 채용확정 이후라도 채용이 취소될 수 있습니다.ㆍ모집분야별로 마감일이 상이할 수 있으니 유의하시길 바랍니다.  [코스닥 상장사] 경영기획 신입 ['경영기획', '전략기획', '경영분석', '경영컨설팅', '사업관리'] 경기 용인시 수지구 신입 · 정규직 대학교(4년)↑",
 "50275785:     [외국계 본사 / 정규

### 채용공고 문서 임베딩

In [61]:
# openai 임베딩
from langchain.embeddings import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002"
    )

response = embeddings.embed_documents(docs)

### 쿼리 임베딩

In [104]:
# 직무 적합 공고 요청
# query = '데이터 분석, AI 모델링, 감성 분석, 개발이 들어간 문서를 찾고싶어.'
# query = 'IT, 빅데이터 산업과 관련된 문서를 찾고싶어.'
# query = 'Python, SQL, R, Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy, Jupyter, Git, Docker, Tableau, MySQL, MongoDB, Hadoop(HDFS), Spark, Google Colab, AWS EC2 & S3, KoNLPy, Word2Vec, Streamlit 기술과 관련한 문서를 찾고싶어.'
# query = '신입, 고려대학교 컴퓨터학과 졸업에 관련된 문서를 찾고싶어'
query = '서울특별시 강남구 관련된 문서를 찾고싶어'

In [105]:
user_embedding = embeddings.embed_query(query)

### 평가 기준별 정합성 계산

* 직무 정합성

In [76]:
# 이후 유저 요청마다 유저 문장만 encode 해서 비교
job_similarity = cosine_similarity([user_embedding], response).flatten()

* 산업/도메인 정합성

In [85]:
domain_similarity = cosine_similarity([user_embedding], response).flatten()

* 기술/스킬 정합성

In [91]:
skill_similarity = cosine_similarity([user_embedding], response).flatten()

* 경력 정합성

In [101]:
career_similarity = cosine_similarity([user_embedding], response).flatten()

* 지역 정합성

In [106]:
place_similarity = cosine_similarity([user_embedding], response).flatten()

In [74]:
df_similarity = pd.read_csv('./data/job_opening_similarity_total.csv')

In [ ]:
df_similarity.head()

,Unnamed: 0.1,Unnamed: 0,rec_idx,jd_text,job_relevance_score,domain_relevance_score,skill_relevance_score,career_relevance_score
0,0,0,50245530,\n채용공고 상세\n[코스닥 상장사] 경영기획본부 신입2007년 12월 11일에 설...,0.666559,0.503435,0.662967,0.544503
1,1,1,50275785,\n채용공고 상세\n,0.513818,0.712619,0.370614,0.638052
2,2,2,50275903,\n채용공고 상세\nNHN Dooray!올인원 협업 도구 두레이와 전자결재/게시판...,0.682043,0.516155,0.659246,0.501009
3,3,3,50254512,\n채용공고 상세\n모집부문 및 자격요건\n 모집부문\n 경력사항\n 담당업무\n자...,0.652005,0.464722,0.674163,0.563210
4,4,4,49874934,\n채용공고 상세\n,0.513818,0.712619,0.370614,0.638052


In [ ]:
df_similarity['job_relevance_score'] = job_similarity

In [86]:
df_similarity['domain_relevance_score'] = domain_similarity

In [92]:
df_similarity['skill_relevance_score'] = skill_similarity

In [102]:
df_similarity['career_relevance_score'] = career_similarity

In [107]:
df_similarity['place_relevance_score'] = place_similarity

### 정합성 저장된 채용공고 CSV 생성

In [113]:
df_similarity.to_csv('./data/job_opening_similarity_total.csv')

# 종합 정합성 계산

In [114]:
df_total_similarity = pd.read_csv('./data/job_opening_similarity_total.csv')

In [116]:
df_total_similarity

,Unnamed: 0,rec_idx,jd_text,job_relevance_score,domain_relevance_score,skill_relevance_score,career_relevance_score,place_relevance_score
0,0,50245530,\n채용공고 상세\n[코스닥 상장사] 경영기획본부 신입2007년 12월 11일에 설...,0.786134,0.807350,0.756113,0.829443,0.792801
1,1,50275785,\n채용공고 상세\n,0.786378,0.798225,0.746665,0.814186,0.812757
2,2,50275903,\n채용공고 상세\nNHN Dooray!올인원 협업 도구 두레이와 전자결재/게시판...,0.783575,0.800743,0.768116,0.802915,0.774334
3,3,50254512,\n채용공고 상세\n모집부문 및 자격요건\n 모집부문\n 경력사항\n 담당업무\n자...,0.810376,0.823632,0.776079,0.830628,0.794356
4,4,49874934,\n채용공고 상세\n,0.793304,0.811125,0.748163,0.809222,0.786207
...,...,...,...,...,...,...,...,...
13848,13848,50185544,\n채용공고 상세\n 구분\n 상세내용\nBusiness\nPO\n(비교대출)\n(...,0.807125,0.812944,0.756402,0.801667,0.804743
13849,13849,50170082,\n채용공고 상세\n\t\n\t 오픈헬스케어(주) ㅣ 전략기획본부 투자팀 - 경력\...,0.793143,0.799999,0.756901,0.817680,0.810015
13850,13850,50090229,\n채용공고 상세\n \n (주)두나미스자산운용\n \n펀드마케팅 \n경력3년이상(...,0.788754,0.791999,0.751709,0.797795,0.797520
13851,13851,50109575,\n채용공고 상세\n재무기획(FP&A) 담당자 채용\n# 모집부문 \n모집부문\n담...,0.790224,0.788250,0.750875,0.806693,0.783643


### 가중치 설정

* 직무 - 0.4
* skill - 0.3
* domain - 0.2
* 경력 - 0.05
* 지역 - 0.05

In [148]:
df_total_similarity['total_score'] = (
    0.4 * df_total_similarity['job_relevance_score'] +
    0.3 * df_total_similarity['skill_relevance_score'] +
    0.2 * df_total_similarity['domain_relevance_score'] +
    0.05 * df_total_similarity['career_relevance_score'] +
    0.05 * df_total_similarity['place_relevance_score']
).round(2)

In [149]:
df_total_similarity = df_total_similarity.sort_values(by='total_score', ascending=False)

In [150]:
df_total_similarity

,Unnamed: 0,rec_idx,jd_text,job_relevance_score,domain_relevance_score,skill_relevance_score,career_relevance_score,place_relevance_score,total_score
3601,3601,50281646,\n채용공고 상세\n모집부문 및 자격요건\n모집부문\n경력구분\n상세내용\n근무지\...,0.877738,0.834617,0.828831,0.836918,0.798263,0.85
3347,3347,50283592,\n채용공고 상세\n알고리즘 개발자 경력직 채용\n서비스 소개\n우리는 데이터를 기...,0.886824,0.847637,0.828697,0.825709,0.801299,0.85
3382,3382,49871635,\n채용공고 상세\n,0.866447,0.836161,0.839015,0.828471,0.798351,0.85
10122,10122,50262494,\n채용공고 상세\n \n \n\n \n㈜유니와이즈솔루션즈\n연구소 AI 및 교육연...,0.862099,0.825562,0.810298,0.844716,0.802122,0.84
3440,3440,50282718,\n채용공고 상세\nAWS기반의 데이터 웨어하우스 개발자 구인최고의 복지로 최고의 ...,0.835622,0.841627,0.838179,0.829977,0.796483,0.84
...,...,...,...,...,...,...,...,...,...
6594,6594,50017077,\n채용공고 상세\n▷ 유니온펫 유통 소개U.P.H(유니온펫 유통)은 UNION O...,0.747360,0.755400,0.717175,0.778952,0.777410,0.74
12026,12026,50170251,\n채용공고 상세\n[벤츠공식딜러사]KCC오토 보증담당자 채용케이씨씨오토(주)는 독...,0.751683,0.756702,0.708293,0.765661,0.773359,0.74
7635,7635,50282694,"\n채용공고 상세\n수원 대형버스 차량정비//대형차량정비(유니버스,그랜버드)2010...",0.743017,0.773026,0.713907,0.785805,0.786272,0.74
2938,2938,50235041,"\n채용공고 상세\nGlobal Freight Services, an affilia...",0.735879,0.753426,0.711729,0.745255,0.747820,0.73


In [146]:
# 임계값 - 0.6 으로 설정

ground_truth = df_total_similarity[df_total_similarity['total_score'] >= 0.83]

In [147]:
ground_truth

,Unnamed: 0,rec_idx,jd_text,job_relevance_score,domain_relevance_score,skill_relevance_score,career_relevance_score,place_relevance_score,total_score
3601,3601,50281646,\n채용공고 상세\n모집부문 및 자격요건\n모집부문\n경력구분\n상세내용\n근무지\...,0.877738,0.834617,0.828831,0.836918,0.798263,0.85
3347,3347,50283592,\n채용공고 상세\n알고리즘 개발자 경력직 채용\n서비스 소개\n우리는 데이터를 기...,0.886824,0.847637,0.828697,0.825709,0.801299,0.85
3839,3839,50085073,\n채용공고 상세\n ...,0.831985,0.847529,0.818600,0.870151,0.828656,0.84
3382,3382,49871635,\n채용공고 상세\n,0.866447,0.836161,0.839015,0.828471,0.798351,0.84
3352,3352,50283525,\n채용공고 상세\n모집부문 및 상세내용공통 자격요건ㆍ학력 : 대졸 이상 (4년)개...,0.860583,0.843414,0.812203,0.846141,0.818561,0.84
...,...,...,...,...,...,...,...,...,...
264,264,50269631,\n채용공고 상세\n\t\n(주)케이티 2025년 KT 대졸신입 ·석사 채용\n네트...,0.834152,0.842661,0.809193,0.832055,0.790394,0.83
3416,3416,50261370,\n채용공고 상세\n(주)마음AI Audio AI Scientist 채용(산업기능/...,0.865443,0.817666,0.811061,0.813810,0.777816,0.83
3977,3977,50275285,\n채용공고 상세\n[네오플] 프로그래밍/기술지원직군 분야별 모집(근무지 : 서울)...,0.832069,0.826035,0.817377,0.842295,0.799448,0.83
3871,3871,50255200,\n채용공고 상세\nAI 스마트팜 시니어 백엔드 개발자\n서비스 소개\n크로프트는 ...,0.854263,0.817538,0.813962,0.809513,0.771800,0.83


In [141]:
len(ground_truth)

2547

In [142]:
ground_truth.to_csv("ground_truth.csv", index=False, encoding="utf-8-sig")